In [4]:
!pip install transformers datasets evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=fb0f170ef1279ee98c13d40cd3a14bf6c96af5b5f7a1ede9851a47db28f95c0f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
from transformers import AutoTokenizer, AutoModelWithLMHead

In [22]:
tokenizer=AutoTokenizer.from_pretrained('T5-base')
model=AutoModelWithLMHead.from_pretrained('T5-base', return_dict=True).to("cuda")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1468: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [15]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")
xsum = load_dataset("xsum", split="test")

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [17]:
xsum

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 11334
})

In [18]:
billsum

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 1237
})

In [24]:
from tqdm import tqdm

In [ ]:
xsum_pred = []
for doc in tqdm(xsum["document"], total=len(xsum["document"])):
  prompt = doc + "\nsummarize:"
  inputs = tokenizer.encode(prompt, return_tensors='pt', max_length=1024, truncation=True).to("cuda")
  output = model.generate(inputs)
  xsum_pred.append(tokenizer.decode(output[0], skip_special_tokens=True))


In [26]:
billsum_pred = []
for doc in tqdm(billsum["text"], total=len(billsum["text"])):
  prompt = doc + "\nsummarize:"
  inputs = tokenizer.encode(prompt, return_tensors='pt', max_length=1024, truncation=True).to("cuda")
  output = model.generate(inputs)
  billsum_pred.append(tokenizer.decode(output[0], skip_special_tokens=True))

100%|██████████| 1237/1237 [08:44<00:00,  2.36it/s]


In [28]:
import evaluate

In [29]:
bleu = evaluate.load("bleu")

In [31]:
print(billsum["summary"])

['Existing property tax law establishes a veterans’ organization exemption under which property is exempt from taxation if, among other things, that property is used exclusively for charitable purposes and is owned by a veterans’ organization.\nThis bill would provide that the veterans’ organization exemption shall not be denied to a property on the basis that the property is used for fraternal, lodge, or social club purposes, and would make specific findings and declarations in that regard. The bill would also provide that the exemption shall not apply to any portion of a property that consists of a bar where alcoholic beverages are served.\nSection 2229 of the Revenue and Taxation Code requires the Legislature to reimburse local agencies annually for certain property tax revenues lost as a result of any exemption or classification of property for purposes of ad valorem property taxation.\nThis bill would provide that, notwithstanding Section 2229 of the Revenue and Taxation Code, no 

In [32]:
results = bleu.compute(predictions=billsum_pred, references=billsum["summary"])

In [34]:
results

{'bleu': 3.0684983523662777e-12,
 'precisions': [0.615909478592142,
  0.26527242707760046,
  0.1529100529100529,
  0.10844674875783107],
 'brevity_penalty': 1.3449635078216716e-11,
 'length_ratio': 0.03841415734361963,
 'translation_length': 17587,
 'reference_length': 457826}

In [36]:
rouge = evaluate.load("rouge")

In [37]:
rouge_results = rouge.compute(predictions=billsum_pred, references=billsum["summary"])

In [38]:
rouge_results

{'rouge1': 0.058342615006815604,
 'rouge2': 0.02447183290110783,
 'rougeL': 0.05105304824893436,
 'rougeLsum': 0.05364050997091621}